In [151]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, Add, Bidirectional, Embedding, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.applications import Xception
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

/var/folders/6w/8_xbb_rs6mq2x4r0cyl6nmbr0000gn/T/ipykernel_77648/42787564.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [152]:
# Loading a text file into memory
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [153]:
# get all imgs with their captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

In [154]:
##Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):

            img_caption.replace("-"," ")
            desc = img_caption.split()

            #converts to lower case
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string

            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions

In [155]:
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    
    return vocab

In [156]:
#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [157]:
dataset_text = "/Users/sarthak/Documents/ICG/Flicker8k_text"
dataset_images = "/Users/sarthak/Documents/ICG/Flicker8k_Dataset"

In [158]:
#we prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))

#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)

#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

#saving each description to file 
save_descriptions(clean_descriptions, "descriptions.txt")

Length of descriptions = 8092
Length of vocabulary =  8763


In [159]:
#Defining Model
def extract_features(directory, model):
        
        features = {}
        for img in tqdm(os.listdir(directory)):
            filename = directory + "/" + img
            image = Image.open(filename)
            image = image.resize((299,299))
            image = np.expand_dims(image, axis=0)
            image = image/127.5
            image = image - 1.0
            feature = model.predict(image)
            features[img] = feature
        return features

In [160]:
model = Xception( include_top=False, pooling='avg' )

In [161]:
# model = EfficientNetB7(include_top=False, pooling='avg', weights='imagenet')

In [162]:
#2048 feature vector
#features = extract_features(dataset_images, model)
#dump(features, open("features.p","wb"))

In [163]:
features = load(open("/Users/sarthak/Documents/ICG/features.p","rb"))

In [164]:
#load the data 
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos

def load_clean_descriptions(filename, photos):   
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        
        words = line.split()
        if len(words)<1 :
            continue
    
        image, image_caption = words[0], words[1:]
        
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions


def load_features(photos):
    #loading all features
    all_features = load(open("/Users/sarthak/Documents/ICG/features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features

In [165]:
filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"

#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions("descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [166]:
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

#creating tokenizer class 
#this will vectorise text corpus
#each integer will represent token in dictionary 

#from keras.preprocessing.text import Tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

In [167]:
# give each word a index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

7577

In [168]:
#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
max_length

32

In [169]:
features['1000268201_693b08cb0e.jpg'][0]

array([0.4734095 , 0.01730891, 0.07334226, ..., 0.08557954, 0.02102289,
       0.23765498], dtype=float32)

In [170]:
def data_generator(descriptions, features, tokenizer, max_length, vocab_size):
    while 1:
        for key, description_list in descriptions.items():
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature, vocab_size)
            # Explicitly return tensors instead of a list
            yield (tf.convert_to_tensor(input_image, dtype=tf.float32),
                   tf.convert_to_tensor(input_sequence, dtype=tf.int32)), tf.convert_to_tensor(output_word, dtype=tf.float32)



# Function to create input-output sequence pairs from the image description
def create_sequences(tokenizer, max_length, desc_list, feature, vocab_size):
    X1, X2, y = list(), list(), list()
    # Walk through each description for the image
    for desc in desc_list:
        # Encode the sequence into a list of integers
        seq = tokenizer.texts_to_sequences([desc])[0]
        # Split one sequence into multiple (input, output) pairs
        for i in range(1, len(seq)):
            # Split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # Pad the input sequence to the max length
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # One-hot encode the output sequence (word)
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Append the data to the lists
            X1.append(feature)  # Image feature
            X2.append(in_seq)   # Sequence (input sequence)
            y.append(out_seq)   # Target word (one-hot encoded)
    return np.array(X1), np.array(X2), np.array(y)


In [171]:
[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length,vocab_size))
a.shape, b.shape, c.shape

(TensorShape([47, 2048]), TensorShape([47, 32]), TensorShape([47, 7577]))

In [172]:

# Define the captioning model
def define_model(vocab_size, max_length):
    # Image features from CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=False)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Merging both models (image and sequence)
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)

    # Final output layer: vocab_size for softmax classification
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    # Tie the model together (image and sequence as inputs, word as output)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Summarize the model and plot
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)

    return model


In [174]:
all_features = load(open("/Users/sarthak/Documents/ICG/features.p", "rb"))
for key, feature in all_features.items():
    print(f"Image: {key}, Feature Shape: {feature.shape}")
    break

Image: 2387197355_237f6f41ee.jpg, Feature Shape: (1, 2048)


In [175]:
# Train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

# Define the model
model = define_model(vocab_size, max_length)

# Training parameters
epochs = 50
steps = len(train_descriptions)

# Add EarlyStopping callback
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

# Train the model with early stopping
for i in range(epochs):
    print(f"Epoch {i+1}/{epochs}")
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1, callbacks=[early_stopping])

    # Save the model after each epoch
    model.save(f"/Users/sarthak/Documents/ICG/Model_saved/fnl_epoch_{i+1}.h5")


Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 7577
Description Length:  32


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_11      │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 32, 256)   │  1,939,712 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 2048)      │          0 │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32, 256)   │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 256)       │    524,544 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 256)       │    525,312 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_86 (Add)        │ (None, 256)       │          0 │ dense_6[0][0],    │
│                     │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 256)       │     65,792 │ add_86[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 7577)      │  1,947,289 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,002,649 (19.08 MB)

 Trainable params: 5,002,649 (19.08 MB)

...

Epoch 46/50
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 415s 69ms/step - accuracy: 0.3784 - loss: 2.6417


Epoch 47/50
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 418s 70ms/step - accuracy: 0.3799 - loss: 2.6433


Epoch 48/50
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 403s 67ms/step - accuracy: 0.3781 - loss: 2.6465


Epoch 49/50
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 400s 67ms/step - accuracy: 0.3768 - loss: 2.6542


Epoch 50/50
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 408s 68ms/step - accuracy: 0.3778 - loss: 2.6519
